In [1]:
import torchvision
import torch
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn
import tqdm as tqdm
import torch.optim as optim
from pytorch_fid import fid_score
import torch.nn.utils as nn_utils


In [2]:
def gradient_critic_score(critic, real, fake, epsilon):

    inter_images = real * epsilon + fake * (1-epsilon)
    inter_images.requires_grad_(True)
    mixed_score = critic(inter_images)

    gradient = torch.autograd.grad(
        inputs = inter_images,
        outputs = mixed_score,
        grad_outputs= torch.ones_like(mixed_score),
        create_graph= True,
        retain_graph=True,
    )[0]

    return gradient

def grad_penalty_l2_norm(gradient):

    gradient = gradient.view(len(gradient), -1)

    gradient_norm = gradient.norm(2,dim=1)

    penalty = torch.mean((gradient_norm - 1) ** 2)

    return penalty

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def get_noise(n_samples, z_dim, device='cpu'):

    return torch.randn(n_samples, z_dim, device=device)


def get_generator_loss(critic_fake_pred):

    gen_loss = -1.0 * torch.mean(critic_fake_pred)

    return gen_loss

def get_critc_loss(critic_fake_pred, critic_real_pred, gp, c_lambda):

    loss = (torch.mean(critic_fake_pred) - torch.mean(critic_real_pred) + c_lambda * gp)

    return loss



In [3]:
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import os

def visualize_fake_images(generator, z_dim, device, num_images=64):
    generator.eval()  
    
    noise = get_noise(num_images, z_dim, device=device)
    with torch.no_grad():
        fake_images = generator(noise).detach().cpu()
    
    generator.train()  

    grid = vutils.make_grid(fake_images, nrow=8, normalize=True)
    plt.figure(figsize=(10, 10))
    plt.imshow(grid.permute(1, 2, 0))
    plt.axis("off")
    plt.title("Generated Images")
    plt.show()



def plot_losses(generator_losses, critic_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(generator_losses, label="Generator Loss")
    plt.plot(critic_losses, label="Critic Loss")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("WGAN-GP Training Losses")
    plt.show()


def calculate_fid_score(generator, z_dim, device, dataloader, epoch, num_images=5000):
    
    gen_dir = f"fid_temp/generated/epoch_{epoch}"
    real_dir = f"fid_temp/real"

    os.makedirs(gen_dir, exist_ok=True)
    os.makedirs(real_dir, exist_ok=True)

    # Save fake images
    generator.eval()
    with torch.no_grad():
        for i in range(num_images):
            z = torch.randn(1, z_dim, 1, 1).to(device)
            fake_img = generator(z).detach().cpu()
            fake_img = (fake_img + 1) / 2
            vutils.save_image(fake_img, os.path.join(gen_dir, f"{i}.png"), normalize=True)

    # Save real images (once is enough)
    if epoch == 0 and not os.listdir(real_dir):
        count = 0
        for real_imgs, _ in dataloader:
            for img in real_imgs:
                if count >= num_images:
                    break
                img = (img + 1) / 2
                vutils.save_image(img, os.path.join(real_dir, f"{count}.png"), normalize=True)
                count += 1
            if count >= num_images:
                break

    generator.train()

    # Calculate FID
    fid_value = fid_score.calculate_fid_given_paths([real_dir, gen_dir], batch_size=64, device=device, dims=2048)
    return fid_value

## Hyper Parameters

*z_dim*:
- 64, 100, 128

*hidden_dim*:
- 64, 128, 256

*learning rate*:
- 1e-4, 1e-5

*n_critic*:
- 3, 5, 7

*c_lambda*:
- 10, 5

*batch_size*:
- 64, 128, 256

In [5]:
c_lambda = 10
n_critic = 5
lr = 0.0001
beta_1 = 0.0
beta_2 = 0.9

hidden_dim = 64
batch_size = 128

image_size = 32
num_channels = 3
z_dim = 128
feature_maps_gen = 64
feature_maps_disc = 64
num_epochs = 10

ngpu = 1

torch.manual_seed(444)



In [6]:

transform = transforms.Compose([
    transforms.Resize(32),                 
    transforms.CenterCrop(32),              
    transforms.ToTensor(),                  # Converts image to [0, 1] float tensor
    transforms.Normalize((0.5, 0.5, 0.5),    # Normalize to [-1, 1]
                         (0.5, 0.5, 0.5))   # Same mean/std for R, G, B channels
])

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)

cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# dataloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batch_size,
#                                          shuffle=True, num_workers=4)

cuda:0


In [7]:
from wgan_gp import Generator
from wgan_gp import Critic
from wgan_gp import CriticDrop
from wgan_gp import CriticSpec

In [8]:
generator = Generator(ngpu =ngpu, z_dim=z_dim, hidden_dim=hidden_dim).to(device)
gen_opt = optim.Adam(generator.parameters(), lr=lr, betas=(beta_1, beta_2))

critic = Critic(im_chan=3, hidden_dim=hidden_dim).to(device)
critic = CriticDrop(im_chan=3, hidden_dim=hidden_dim).to(device)
critic = CriticSpec(im_chan=3, hidden_dim=hidden_dim).to(device)

crit_opt = optim.Adam(critic.parameters(), lr=lr, betas=(beta_1, beta_2))

generator = generator.apply(weights_init)
critic = critic.apply(weights_init)

In [17]:
import wandb
wandb.login()

sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'FID',
      'goal': 'minimize'  
    },
    'parameters': {
        'z_dim': {'values': [64, 100, 128]},
        'hidden_dim': {'values': [64, 128, 256]},
        'lr': {'values': [1e-4, 1e-5]},
        'n_critic': {'values': [3, 5, 7]},
        'c_lambda': {'values': [5, 10]},
        'batch_size': {'values': [64, 128, 256]},
        'epochs': {'value': 10}  # You can increase this later
    }
}

In [18]:
sweep_id = wandb.sweep(sweep_config, project='gan_spec_drop-sweep_final')

Create sweep with ID: jlyx2hte
Sweep URL: https://wandb.ai/jhanshaw-uc-san-diego/gan_spec_drop-sweep_final/sweeps/jlyx2hte


In [19]:
def sweep_train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Rebuild models with given config
        generator = Generator(ngpu=1, z_dim=config.z_dim, hidden_dim=config.hidden_dim).to(device)
        
        #critic = Critic(hidden_dim=config.hidden_dim).to(device)
        #critic = CriticSpec(hidden_dim=config.hidden_dim).to(device)
        critic = CriticDrop(hidden_dim=config.hidden_dim).to(device)

        generator.apply(weights_init)
        critic.apply(weights_init)

        gen_opt = torch.optim.Adam(generator.parameters(), lr=config.lr, betas=(0.5, 0.999))
        crit_opt = torch.optim.Adam(critic.parameters(), lr=config.lr, betas=(0.5, 0.999))

        dataloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=config.batch_size,
                                         shuffle=True, num_workers=4)

        generator_losses = []
        critic_losses_across_repeats = []
        fid_scores = []

        for epoch in range(config.epochs):
            for real, _ in dataloader:
                cur_batch_size = len(real)
                real = real.to(device)

                mean_critic_loss = 0

                for _ in range(config.n_critic):
                    crit_opt.zero_grad()

                    noise = get_noise(cur_batch_size, config.z_dim, device)
                    fake = generator(noise)

                    critic_real = critic(real)
                    critic_fake = critic(fake.detach())

                    epsilon = torch.rand(cur_batch_size, 1, 1, 1, device=device, requires_grad=True)
                    grad = gradient_critic_score(critic, real, fake.detach(), epsilon)
                    gp = grad_penalty_l2_norm(grad)

                    crit_loss = get_critc_loss(critic_fake, critic_real, gp, config.c_lambda)
                    mean_critic_loss += crit_loss.item() / config.n_critic

                    crit_loss.backward()
                    crit_opt.step()

                critic_losses_across_repeats.append(mean_critic_loss)

                # Generator update
                gen_opt.zero_grad()
                noise2 = get_noise(cur_batch_size, config.z_dim, device)
                fake2 = generator(noise2)
                gen_loss = get_generator_loss(critic(fake2))
                gen_loss.backward()
                gen_opt.step()

                generator_losses.append(gen_loss.item())

            # Every epoch: log FID
            fid = calculate_fid_score(generator, config.z_dim, device, dataloader, epoch)
            fid_scores.append(fid)

            wandb.log({
                "epoch": epoch,
                "gen_loss": gen_loss.item(),
                "crit_loss": mean_critic_loss,
                "FID": fid
            })

        print(f"Final FID: {fid_scores[-1]:.2f}")

In [20]:
wandb.agent(sweep_id, function=sweep_train, count=10)

wandb: Agent Starting Run: vc1vtq66 with config:
wandb: 	batch_size: 64
wandb: 	c_lambda: 5
wandb: 	epochs: 10
wandb: 	hidden_dim: 64
wandb: 	lr: 1e-05
wandb: 	n_critic: 7
wandb: 	z_dim: 64


100%|██████████| 79/79 [00:43<00:00,  1.81it/s]


Final FID: 263.51


FID,▅▆▃█▅▅▄▂▁▅
crit_loss,▃▁▃▂▅▄█▆█▆
epoch,▁▂▃▃▄▅▆▆▇█
gen_loss,▇██▆▅▄▄▄▂▁
FID,263.51096
crit_loss,-0.46394
epoch,9
gen_loss,-4.11275


wandb: Agent Starting Run: 5cz8hukt with config:
wandb: 	batch_size: 128
wandb: 	c_lambda: 5
wandb: 	epochs: 10
wandb: 	hidden_dim: 64
wandb: 	lr: 0.0001
wandb: 	n_critic: 7
wandb: 	z_dim: 128


...

Run enn2ztja errored:
Traceback (most recent call last):
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\jacob\AppData\Local\Temp\ipykernel_23704\3285193204.py", line 64, in sweep_train
    fid = calculate_fid_score(generator, config.z_dim, device, dataloader, epoch)
  File "C:\Users\jacob\AppData\Local\Temp\ipykernel_23704\450502378.py", line 67, in calculate_fid_score
    fid_value = fid_score.calculate_fid_given_paths([real_dir, gen_dir], batch_size=64, device=device, dims=2048)
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\pytorch_fid\fid_score.py", line 261, in calculate_fid_given_paths
    m2, s2 = compute_statistics_of_path(paths[1], model, batch_size,
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\pytorch_fid\fid_score.py", line 243, in compute_statistics_of_path
    m, s = calculate_activation_statistics(files, model, batch_size,


 15%|█▌        | 12/79 [00:09<00:52,  1.27it/s]
Traceback (most recent call last):
  File "C:\Users\jacob\AppData\Local\Temp\ipykernel_23704\3285193204.py", line 64, in sweep_train
    fid = calculate_fid_score(generator, config.z_dim, device, dataloader, epoch)
  File "C:\Users\jacob\AppData\Local\Temp\ipykernel_23704\450502378.py", line 67, in calculate_fid_score
    fid_value = fid_score.calculate_fid_given_paths([real_dir, gen_dir], batch_size=64, device=device, dims=2048)
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\pytorch_fid\fid_score.py", line 261, in calculate_fid_given_paths
    m2, s2 = compute_statistics_of_path(paths[1], model, batch_size,
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\pytorch_fid\fid_score.py", line 243, in compute_statistics_of_path
    m, s = calculate_activation_statistics(files, model, batch_size,
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\pytorch_fid\fid_score.py", line 228, in cal

FID,▁▆█
crit_loss,▁▇█
epoch,▁▅█
gen_loss,▁█▃
FID,304.54113
crit_loss,-1.19607
epoch,2
gen_loss,-0.92109


Run n9dx9bxm errored:
Traceback (most recent call last):
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\jacob\AppData\Local\Temp\ipykernel_23704\3285193204.py", line 64, in sweep_train
    fid = calculate_fid_score(generator, config.z_dim, device, dataloader, epoch)
  File "C:\Users\jacob\AppData\Local\Temp\ipykernel_23704\450502378.py", line 67, in calculate_fid_score
    fid_value = fid_score.calculate_fid_given_paths([real_dir, gen_dir], batch_size=64, device=device, dims=2048)
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\pytorch_fid\fid_score.py", line 261, in calculate_fid_given_paths
    m2, s2 = compute_statistics_of_path(paths[1], model, batch_size,
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\pytorch_fid\fid_score.py", line 243, in compute_statistics_of_path
    m, s = calculate_activation_statistics(files, model, batch_size,


message_loop has been closed
Traceback (most recent call last):
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\wandb\sdk\lib\sock_client.py", line 208, in _read_packet_bytes
    data = self._sock.recv(self._bufsize)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\wandb\sdk\interface\router_sock.py", line 27, in _read_message
    return self._sock_client.read_server_response(timeout=1)
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\wandb\sdk\lib\sock_client.py", line 235, in read_server_response
    data = self._read_packet_bytes(timeout=timeout)
  File "c:\Users\jacob\anaconda3\envs\mario_env_2\lib\site-packages\wandb\sdk\lib\sock_client.py", line 212, in _read_packet_bytes
    raise SockClientClosedError from e
wandb.sdk